In [26]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
from datetime import datetime
import time
import os
import tensorflow as tf
import csv
import pandas as pd
from pandas import Series, DataFrame
from numpy import array
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3,4"
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


INPUT_NODE = 2304
OUTPUT_NODE = 7
BATCH_SIZE = 1024
LEARNING_RATE_BASE = 0.05
LEARNING_RATE_DECAY = 0.995
REGULARIZER = 0.0001
STEPS = 100000
MOVING_AVERAGE_DECAY = 0.99
TEST_INTERVAL_SECS = 3
MODEL_SAVE_PATH="check_point"
MODEL_NAME="data_model"



def get_weight(shape, regularizer):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1,mean=0))
    #损失函数loss含正则化regularization
    if regularizer != None: tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w


def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b


def conv2d(x, W):
    # stride[1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] =1，意思是不对样本个数和channel进行卷积
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")  # padding="SAME"用零填充边界


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


def forward(x, regularizer):
 
    x = tf.reshape(x, [-1, 48, 48, 1])
    
    ## convl layer ##
    W_conv1 = get_weight([5,5,1,128],None) # kernel 5*5, channel is 1, out size 128
    b_conv1 = get_bias([128])
    h_conv1 = tf.nn.relu(conv2d(x,W_conv1) + b_conv1)  
    h_pool1 = max_pool_2x2(h_conv1)                          
    
    ## conv2 layer ##
    W_conv2 = get_weight([5,5,128,64],None) # kernel 5*5, in size 128, out size 64
    b_conv2 = get_bias([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)  
    h_pool2 = max_pool_2x2(h_conv2)                         
    
    ## conv3 layer ##
    W_conv3 = get_weight([5,5,64,32],None) # kernel 5*5, in size 64, out size 32
    b_conv3 = get_bias([32])
    h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3) + b_conv3)  
    h_pool3 = max_pool_2x2(h_conv3)                         
    
    ## funcl layer ##
    W_fc1 = get_weight([1*6*6*32, 576], regularizer)
    b_fc1 = get_bias([576])
    
    h_pool2_flat = tf.reshape(h_pool3, [-1, 1*6*6*32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob = 0.5)
    
    ## func2 layer ##
    W_fc2 = get_weight([576, 128], regularizer)
    b_fc2 = get_bias([128])
    y1 = tf.nn.relu(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
    
    W_fc3 = get_weight([128, OUTPUT_NODE], regularizer)
    b_fc3 = get_bias([OUTPUT_NODE])
    y = (tf.matmul(y1,W_fc3)+b_fc3)
    tf.add_to_collection('pred_network', y)  # 用于加载模型获取要预测的网络结构
    
    return W_fc1,b_fc1,h_fc1,W_fc2,b_fc2,y



def predict(new_data):

    keep_prob = tf.placeholder(tf.float32)
  
    with tf.Graph().as_default() as tg:
        x = tf.placeholder(tf.float32, [None, INPUT_NODE])
        w1, b1, y1, w2, b2, y = forward(x, None)


        preValue = tf.argmax(y, 1)

        print("type of prevalue = ",type(preValue))

        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        with tf.Session(config=config) as sess:
            init_op = tf.global_variables_initializer()
            sess.run(init_op)
            ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                xs = new_data[:, :INPUT_NODE]
                preValue = sess.run(preValue, feed_dict={x: xs})
                print(type(preValue),len(preValue))
                
                preValue = preValue + 1
                
                print(preValue)
                data1 = DataFrame(preValue)
                data1.to_csv('predict_data.csv')
                print("finish!!!")

            else:
                print("No checkpoint file found")
                return -1


def main():
    

    print("predict!")
    data = pd.read_csv("predict.csv",low_memory=False).values
    data = data.astype(np.float32)  # change to numpy array and float32
    data = data / 255.0
    print("finish test!")

    print("数组元素总数：",data.size)      #打印数组尺寸，即数组元素总数  
    print("数组形状：",data.shape)         #打印数组形状 
    print("数组的维度数目",data.ndim)      #打印数组的维度数目

    predict(data)


if __name__ == '__main__':

    main()

    


predict!
finish test!
数组元素总数： 488448
数组形状： (212, 2304)
数组的维度数目 2
type of prevalue =  <class 'tensorflow.python.framework.ops.Tensor'>
INFO:tensorflow:Restoring parameters from check_point/data_model-29290


ResourceExhaustedError: OOM when allocating tensor with shape[212,128,48,48] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node Conv2D (defined at <ipython-input-26-2612b44238ed>:51)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, Variable/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node ArgMax/_27}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_95_ArgMax", tensor_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
data = pd.read_csv("test.csv",low_memory=False).values
data = data.astype(np.float32)  # change to numpy array and float32
data_x = data[:,:2304]
data_y = data[:,2304:]
for ix in range(7178):
    plt.figure(ix)
    plt.imshow(data_x[ix].reshape((48, 48)), interpolation='none', cmap='gray')
    plt.title(data_y[ix])
    picname = str(ix) + '.png'
    plt.savefig(picname)
plt.show()

/home/multiphase/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


IndexError: index 7177 is out of bounds for axis 0 with size 7177